In [1]:
!pip install osmnx torch_geometric

In [3]:
import os
import time
import shutil
import numpy as np
import os.path as osp
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors # Dùng cái này để tìm k-NN

# Import PyG dependencies
import torch_geometric
from torch_geometric.data import Data, InMemoryDataset, download_url
from torch_geometric.nn import GATv2Conv, GCNConv

# Thiết lập thiết bị
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ==============================================================================
# 1. TRAVELDataset (Giữ nguyên)
# ==============================================================================

def read_npz(path):
    with np.load(path, allow_pickle=True) as f:
        return parse_npz(f)

def parse_npz(f):
    x = torch.from_numpy(f['x']).to(torch.float)
    coords = torch.from_numpy(f['coordinates']).to(torch.float)
    edge_attr = torch.from_numpy(f['edge_attr']).to(torch.float)
    occur_labels = torch.from_numpy(f['occur_labels']).to(torch.long)
    edge_index = torch.from_numpy(f['edge_index']).to(torch.long).t().contiguous()
    try:
        severity_labels = torch.from_numpy(f['severity_8labels']).to(torch.long)
    except:
        severity_labels = torch.zeros_like(occur_labels)

    return Data(x=x, y=occur_labels, severity_labels=severity_labels, 
                edge_index=edge_index, edge_attr=edge_attr, coords=coords)

class TRAVELDataset(InMemoryDataset):
    url = 'https://github.com/baixianghuang/travel/raw/main/TAP-city/{}.npz'

    def __init__(self, root, name, transform=None, pre_transform=None):
        self.name = name.lower()
        super().__init__(root, transform, pre_transform)
        try:
            self.data, self.slices = torch.load(self.processed_paths[0], weights_only=False)
        except TypeError:
            self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_dir(self): return osp.join(self.root, self.name, 'raw')
    @property
    def processed_dir(self): return osp.join(self.root, self.name, 'processed')
    @property
    def raw_file_names(self): return f'{self.name}.npz'
    @property
    def processed_file_names(self): return 'data.pt'

    def download(self):
        download_url(self.url.format(self.name), self.raw_dir)

    def process(self):
        data = read_npz(self.raw_paths[0])
        if self.pre_transform is not None:
            data = self.pre_transform(data)
        torch.save(self.collate([data]), self.processed_paths[0])

# ==============================================================================
# 2. Xây dựng đồ thị k-NN (Giải pháp triệt để cho vấn đề RAM)
# ==============================================================================

def construct_knn_graphs(data, k=15, sigma=1.0):
    """
    Sử dụng k-Nearest Neighbors để giới hạn số lượng cạnh.
    Đảm bảo bộ nhớ không bao giờ bị tràn.
    k: Số lượng hàng xóm tối đa cho mỗi nút (vd: 10, 15, 20).
    """
    print(f"Constructing k-NN graphs (k={k}) on CPU...")
    
    coords = data.coords.cpu().numpy()
    num_nodes = len(coords)
    
    # Sử dụng Sklearn NearestNeighbors (Rất nhanh và nhẹ RAM)
    # algorithm='kd_tree' giúp tìm kiếm nhanh
    nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='kd_tree').fit(coords)
    distances, indices = nbrs.kneighbors(coords)
    
    # indices: [N, k+1] chứa index của các nút hàng xóm
    # Cột 0 là chính nó (khoảng cách 0), ta bỏ qua hoặc giữ tùy ý. Ở đây ta bỏ cột 0.
    knn_idx = indices[:, 1:] # [N, k]
    knn_dist = distances[:, 1:] # [N, k]
    
    # Flatten để tạo edge_index
    source_nodes = np.repeat(np.arange(num_nodes), k)
    target_nodes = knn_idx.flatten()
    dist_flat = knn_dist.flatten()
    
    # --- Tạo A1 (Physical Graph) ---
    # Tính trọng số cạnh dựa trên khoảng cách
    # weight = exp(-d^2 / sigma^2)
    weights_a1 = np.exp(-(dist_flat**2) / (sigma**2))
    
    # Tạo edge_index tensor
    edge_index_1 = torch.tensor([source_nodes, target_nodes], dtype=torch.long)
    
    # --- Tạo A2 (Accident Graph) ---
    # Trọng số bị ảnh hưởng bởi intensity t của nút đích (target_nodes)
    y_np = data.y.cpu().numpy()
    t_vals = np.where(y_np > 0, 1.2, 1.0) # [N]
    
    # Lấy t tương ứng với mỗi cạnh (dựa vào target node)
    t_targets = t_vals[target_nodes]
    
    weights_a2 = np.exp(-(dist_flat**2) / ((sigma**2) * t_targets))
    
    # Với k-NN, cấu trúc kết nối (topology) của A1 và A2 là giống nhau (đều là k hàng xóm),
    # chỉ khác nhau ở trọng số (weights) dùng trong Attention.
    # Trong mô hình GAT, ta chỉ cần truyền cấu trúc edge_index. 
    # (Để đơn giản và tiết kiệm, ta dùng chung edge_index, GAT sẽ tự học trọng số mới).
    
    # Chuyển sang device
    edge_index = edge_index_1.to(device)
    
    print(f" -> Total Edges: {edge_index.size(1)} (Fixed size: {num_nodes} * {k})")
    return edge_index

# ==============================================================================
# 3. Mô hình Sparse AI-GFACN
# ==============================================================================

class SparseAIGFACN(nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes, dropout=0.5):
        super(SparseAIGFACN, self).__init__()
        
        # Embedding
        self.embedding = nn.Sequential(
            nn.Linear(num_features, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        # Dual Graph Processing
        # Ta dùng 2 lớp GAT riêng biệt để học 2 góc nhìn khác nhau trên cùng tập hàng xóm
        self.gat1 = GATv2Conv(hidden_dim, hidden_dim, heads=1, concat=True, dropout=dropout)
        self.gat2 = GATv2Conv(hidden_dim, hidden_dim, heads=1, concat=True, dropout=dropout)
        
        # Fusion
        self.gate_w = nn.Linear(2*hidden_dim, 1)
        
        # Decoder
        self.gcn = GCNConv(hidden_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, num_classes)
        self.dropout = dropout

    def forward(self, x, edge_index):
        h = self.embedding(x)
        h = F.dropout(h, p=self.dropout, training=self.training)
        
        # Nhánh 1: Physical View (GAT sẽ tự học trọng số attention dựa trên features)
        h1 = self.gat1(h, edge_index) 
        h1 = F.elu(h1)
        
        # Nhánh 2: Accident View
        # Lưu ý: Trong bản gốc, A1 và A2 khác nhau về trọng số khởi tạo. 
        # Ở đây GATv2 đủ mạnh để tự tìm trọng số tối ưu khác nhau cho 2 nhánh.
        h2 = self.gat2(h, edge_index) 
        h2 = F.elu(h2)
        
        # Gated Fusion
        combined = torch.cat([h1, h2], dim=1)
        z = torch.sigmoid(self.gate_w(combined))
        h_fused = z * h1 + (1 - z) * h2
        
        # Final Processing
        h_final = self.gcn(h_fused, edge_index)
        h_final = F.relu(h_final)
        h_final = F.dropout(h_final, p=self.dropout, training=self.training)
        
        out = self.fc_out(h_final)
        return F.log_softmax(out, dim=1)

# ==============================================================================
# 4. Training Loop
# ==============================================================================

def train_test_split_stratify(dataset, train_ratio, val_ratio, class_num):
    labels = dataset[0].y
    train_mask = torch.zeros(size=labels.shape, dtype=torch.bool)
    val_mask = torch.zeros(size=labels.shape, dtype=torch.bool)
    test_mask = torch.zeros(size=labels.shape, dtype=torch.bool)
    
    for i in range(class_num):
        idx = np.argwhere(labels.cpu().numpy() == i).flatten()
        np.random.shuffle(idx)
        split1 = int(len(idx) * train_ratio)
        split2 = split1 + int(len(idx) * val_ratio)
        train_mask[idx[:split1]] = True
        val_mask[idx[split1:split2]] = True
        test_mask[idx[split2:]] = True
    return train_mask, val_mask, test_mask

def run_experiment(city_name='los_angeles_ca', epochs=50):
    root_path = './data_travel'
    
    print(f"Loading data for {city_name}...")
    dataset = TRAVELDataset(root_path, city_name)
    data = dataset[0]
    
    # Giải phóng bộ nhớ không cần thiết
    data.edge_attr = None 
    
    class_num = dataset.num_classes
    train_mask, val_mask, test_mask = train_test_split_stratify(dataset, 0.6, 0.2, class_num)
    
    # Chuẩn hóa
    sc = MinMaxScaler()
    data.x = torch.tensor(sc.fit_transform(data.x.numpy()), dtype=torch.float)
    
    # Xây dựng đồ thị k-NN (Thay đổi k nếu cần, k càng nhỏ càng nhẹ)
    # k=10 là con số rất an toàn cho mọi RAM
    edge_index = construct_knn_graphs(data, k=10, sigma=1.0)
    
    # Move Data to GPU
    data = data.to(device)
    
    # Model
    model = SparseAIGFACN(num_features=dataset.num_features, hidden_dim=64, num_classes=class_num).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
    
    print("-" * 85)
    print(f"{'Epoch':<5} | {'Loss':<8} | {'Acc':<8} | {'MAE':<8} | {'RMSE':<8} | {'PCC':<8} | {'Time':<8}")
    print("-" * 85)
    
    for epoch in range(1, epochs + 1):
        t_start = time.time()
        
        model.train()
        optimizer.zero_grad()
        
        output = model(data.x, edge_index)
        loss = F.nll_loss(output[train_mask], data.y[train_mask])
        
        loss.backward()
        optimizer.step()
        
        # Eval
        model.eval()
        with torch.no_grad():
            output = model(data.x, edge_index)
            probs = torch.exp(output)[:, 1]
            preds = output.max(1)[1]
            
            y_true = data.y[test_mask].cpu().numpy()
            y_pred = preds[test_mask].cpu().numpy()
            y_prob = probs[test_mask].cpu().numpy()
            
            acc = accuracy_score(y_true, y_pred)
            mae = mean_absolute_error(y_true, y_prob)
            rmse = np.sqrt(mean_squared_error(y_true, y_prob))
            
            if np.std(y_prob) < 1e-9: pcc = 0.0
            else:
                pcc_val, _ = pearsonr(y_true, y_prob)
                pcc = 0.0 if np.isnan(pcc_val) else pcc_val
        
        if epoch % 5 == 0 or epoch == 1:
            print(f"{epoch:<5} | {loss.item():<8.4f} | {acc:<8.4f} | {mae:<8.4f} | {rmse:<8.4f} | {pcc:<8.4f} | {time.time()-t_start:<8.4f}s")

if __name__ == "__main__":
    if not osp.exists('./data_travel'):
        os.makedirs('./data_travel')
    
    # Dọn dẹp cache
    torch.cuda.empty_cache()
    
    try:
        run_experiment('los_angeles_ca', epochs=200)
    except Exception as e:
        print(f"Error: {e}")

Using device: cuda
Loading data for los_angeles_ca...
Constructing k-NN graphs (k=10) on CPU...
 -> Total Edges: 492510 (Fixed size: 49251 * 10)
-------------------------------------------------------------------------------------
Epoch | Loss     | Acc      | MAE      | RMSE     | PCC      | Time    
-------------------------------------------------------------------------------------
1     | 0.7834   | 0.8627   | 0.4921   | 0.4922   | 0.0169   | 0.1358  s
5     | 0.4111   | 0.8698   | 0.1902   | 0.3403   | 0.0121   | 0.1037  s
10    | 0.4160   | 0.8698   | 0.2260   | 0.3351   | 0.1003   | 0.1046  s
15    | 0.4136   | 0.8698   | 0.2622   | 0.3366   | 0.2083   | 0.1037  s
20    | 0.3902   | 0.8698   | 0.1898   | 0.3328   | 0.3037   | 0.1038  s
25    | 0.3557   | 0.8700   | 0.2395   | 0.3139   | 0.4223   | 0.1040  s
30    | 0.3175   | 0.8847   | 0.1626   | 0.2996   | 0.4765   | 0.1045  s
35    | 0.3129   | 0.8795   | 0.1876   | 0.3005   | 0.4661   | 0.1038  s
40    | 0.3122   | 0.8796  